# Polynomial Functions

In [ ]:
import numpy as np
import itertools



#Easier to read polynomial function
def poly(lst):  
    return np.poly1d(lst)

#To extract the degree of the polynomial, use:
def deg(poly):  
    return poly.order

#To extract the leading coefficient of the polynomial:
def leadingcoff(poly):
    return int(poly.c[0])

#x^k generator
#INPUT: k
#OUTPUT: x^k
def monomial_k(k):
    lst = []
    for i in range(0, k+1):  
        if i != 0:
            lst.append(0)    
        else:
            lst.append(1)        
    return poly(lst)

#Takes polynomials from Z[x] into F_p[x]
#INPUT: polynomial in Z[x]
#OUTPUT: polynomial in F_p[x]
def ZtoFq(poly, p):
    
    new_coeff = []
    d = deg(poly)
    
    for k in range(0, d + 1):
        
        new_coeff.append((poly[d - k] % p))  
        
    return np.poly1d(new_coeff)

#I'm stupid. I'm so stupid
#INPUT: f, g polynomials in Z/pZ[x]
#OUTPUT: the image of f under the quotient map into Z/pZ[x]/(g)
def fasterquotientmap(f, g, p):
    
    return polydiv(f, g, p)[1]

#Generates list of all possible lists of polynomial coefficients in Z/pZ with degree strictly less than d
def lstofpolynomials(p, d):
    
    lst = []
    
    for coeffs in itertools.product(range(p), repeat = d):
        lst.append(coeffs)

    return lst

#Performs polynomial long division
def polydiv(f, g, p):
    f = ZtoFq(f, p)
    g = ZtoFq(g, p)

    if g == poly([0]):
        return "I am so dissappointed"

    q = poly([0])
    r = f

    while (r != poly([0])) and (deg(r) >= deg(g)):
        # This is what we subtract by during long division
        coeff = (leadingcoff(r) * inverse(int(leadingcoff(g)), p)) % p
        t = coeff * monomial_k(deg(r) - deg(g))

        q_new = q + t
        r_new = r - t * g

        q = ZtoFq(q_new, p)
        r = ZtoFq(r_new, p)

    return (q, r)



In [75]:
polydiv(poly([1, 3, 4, 2, 1]), poly([1,2]), 5)

(poly1d([1., 1., 2., 3.]), poly1d([0.]))

In [76]:
fasterquotientmap(poly([1,3,4,2,1]), poly([1,2]), 5)

poly1d([0.])

In [38]:
2*poly([1,2,3])

poly1d([2, 4, 6])

# Finite Field Tools

In [156]:
import math

# Applies the Euclidean Algorithm to compute the gcd of polynomials f and g in Z/pZ[x]
def gcdfinder(f, g, p):
    r0 = f
    r1 = g
    u0 = poly([1])
    v0 = poly([0])
    u1 = poly([0])
    v1 = poly([1])

    while (r1 != poly([0])):
        quot = polydiv(r0, r1, p)[0]

        R = ZtoFq(-quot * r1 + r0, p)
        r0 = r1
        r1 = R

        U = ZtoFq(-quot * u1 + u0, p)
        u0 = u1
        u1 = U

        V = ZtoFq(-quot * v1 + v0, p)
        v0 = v1
        v1 = V

    return (u0, v0, r0)

#Computes the inverse of x in Z/pZ
def inverse(x, p):
    inv = 1
    for k in range(1, p):
        if (k*x % p == 1):
            inv = k
    return inv

#INPUT: polynomial g in Z/pZ[x]
#OUTPUT: a monic polynomial g' such that (g) = (g') in Z/pZ[x]
def makemonic(g, p):
    f = inverse(leadingcoff(g), p)*g
    return ZtoFq(f, p)

#Brute force irreducibility checker for polynomials in Z/pZ[x]
def irreducible(f, p):
    
    d = deg(f)
        
    for poly in lstofpolynomials(p, d):
        
        if deg(np.poly1d(poly)) == 0:
            
            continue
        
        if polydiv(f, np.poly1d(poly), p)[1] == np.poly1d([0]):
            
            return False
    
    return True

In [163]:
irreducible(poly([1, 1,3]), 5)

False

In [161]:
polydiv(poly([2, 2, 12, 0]), poly([1, 0]), 13)[1] == np.poly1d([0])

True

In [148]:
lstofpolynomials(13, 2)

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (2, 12),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (3, 10),
 (3, 11),
 (3, 12),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 10),
 (4, 11),
 (4, 12),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 7),
 (5, 8),
 (5, 9),
 (5, 10),
 (5, 11),
 (5, 12),
 (6, 0),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5),
 (6, 6),
 (6, 7),
 (6, 8),
 (6, 9),
 (6, 10),
 (6, 11),
 (6, 12),
 (7, 0),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (7, 7),
 (7, 8),
 (7, 9),
 (7, 10),
 (7, 11),
 (7, 12),
 (8, 0),
 (8, 1),
 (8, 2),
 (8, 3),
 (8,

# Random Generator Tools

In [164]:
import random as rnd


#Generates a random polynomial of degree d in Z/pZ[x]
def randompoly(d, p):
    
    lst = [rnd.randint(1, p-1)]
    
    for l in range(1, d+1):
        
        lst.append(rnd.randint(0, p-1))
    
    return np.poly1d(lst)

#Generates a random irreducible polynomial of degree d in Z/pZ[x]
def randomirreducible(d, p):
    
    f = randompoly(d, p)
    
    while irreducible(f, p) == False:
        
        f = randompoly(d, p)
    
    return f


In [106]:
randomirreducible(4, 7)

poly1d([2, 0, 2, 5, 2])

# Legendre Symbol

In [107]:
from sympy.ntheory import legendre_symbol

def legendresymbol(f, h, p):
    
    legsym = 1
    h = makemonic(h, p)
    
    while True:
        
        f = fasterquotientmap(f, h, p)
        
        if (f == poly([0])):
            return 0
        
        c = leadingcoff(f)
        
        if (deg(f) == 0):    
            return (legendre_symbol(int(c), p)**(deg(h)))*legsym
        
        f = ZtoFq(inverse(c, p)*f, p)
        
        if (((p-1)/2) % 2 == 1) and (deg(f) % 2 == 1) and (deg(h) % 2 == 1):
            
            legsym = -legendre_symbol(int(c), p)**(deg(h))*legsym
            
        else:
            
            legsym = legendre_symbol(int(c), p)**(deg(h))*legsym
            
        G = h
        h = f
        f = G
   

# Brute Force Legendre Symbol

In [108]:
#Computes the set of squares in the quotient ring F_p[T]/(g) where deg(g) = d (Only works for q = p prime and odd)
#This code also requires that g is monic or else the image under the quotient map will get... weird
#For convenience, g will be become a monic polynomial that generates the same ideal as g
def lstofsquares(g, p):
    
    g = makemonic(g, p)
    
    d = deg(g)
    
    lst = lstofpolynomials(p, d)
    
    sqlst = []
    
    for poly in lst:
        
        f = fasterquotientmap(np.poly1d(poly)*np.poly1d(poly), g, p)

        sqlst.append(list(f.c))
    
    #This line of code erases duplicate entries in the list of squares
    sqlst = [k for k,v in itertools.groupby(sorted(sqlst))]
        
    return sqlst

#WARNING: This method only works for irreducible g.
#Remember that the Legendre symbol defined as f = square mod g works for g irreducible.
#When g is not, we are working with the Jacobi symbol which does not have the same residue meaning as the
#Legendre symbol! i.e. (f/g) = 1 does not imply that f = square mod g
def bruteforcelegendrecomputation(f, g, p):
    
    g = makemonic(g, p)
    f = fasterquotientmap(f, g, p)
    f_coeff = list(f.c)
    
    if f_coeff == [0]:
        
        return 0
    
    if (f_coeff in lstofsquares(g, p)) == True:
        
        return 1
    
    if (f_coeff in lstofsquares(g, p)) == False:
        
        return -1
        

# Legendre Symbol Verifier

In [165]:
#Performs n random tests of the Legendre symbol between two polynomials of at most degree d
#Compares outputs of Legendresymbol function and bruteforcelegendrecomputation function
def testlegendresymbol(n, d):
    
    k = 0
    primes = [3, 5, 7, 11, 13, 17, 19, 23]
    
    while k < n:
        
        p = primes[rnd.randint(0, 7)]
        
        deg1 = rnd.randint(0, d)
        deg2 = rnd.randint(1, d)
        
        f = randompoly(deg1, p)
        print(f)
        
        g = randomirreducible(deg2, p)
        print(g)
        print(irreducible(g, p))
        
        print(p)
        print(legendresymbol(f, g, p))
        print(bruteforcelegendrecomputation(f, g, p))
        print(legendresymbol(f, g, p) == bruteforcelegendrecomputation(f, g, p))
            
        k = k + 1
    

In [ ]:
testlegendresymbol(30, 6)

 
1
 
2 x
True
3
1
1
True
   4     3     2
7 x + 5 x + 9 x + 12 x + 10
    3     2
15 x + 6 x + 9 x + 10
True
17
1
1
True
   5     4     3     2
8 x + 3 x + 9 x + 6 x + 1 x + 6
 
2 x + 9
True
13
-1
-1
True
 
4
    3     2
10 x + 3 x + 12 x + 11
True
13
1
1
True
   4     2
4 x + 2 x + 4 x + 3
   3     2
2 x + 1 x + 4 x + 1
True
5
1
1
True
   2
2 x + 2 x + 4
 
1 x + 2
True
7
1
1
True
   5     4     3     2
1 x + 9 x + 2 x + 8 x + 7 x + 7
   3     2
5 x + 3 x + 8 x + 5
True
11
-1
-1
True
 
13 x + 2
   5      4      3      2
7 x + 14 x + 15 x + 10 x + 8 x + 17
True
23
-1
